# T1543.002 - Systemd Service
Adversaries may create or modify systemd services to repeatedly execute malicious payloads as part of persistence. Systemd is a system and service manager commonly used for managing background daemon processes (also known as services) and other system resources.(Citation: Linux man-pages: systemd January 2014) Systemd is the default initialization (init) system on many Linux distributions replacing legacy init systems, including SysVinit and Upstart, while remaining backwards compatible.  

Systemd utilizes unit configuration files with the `.service` file extension to encode information about a service's process. By default, system level unit files are stored in the `/systemd/system` directory of the root owned directories (`/`). User level unit files are stored in the `/systemd/user` directories of the user owned directories (`$HOME`).(Citation: lambert systemd 2022) 

Inside the `.service` unit files, the following directives are used to execute commands:(Citation: freedesktop systemd.service)  

* `ExecStart`, `ExecStartPre`, and `ExecStartPost` directives execute when a service is started manually by `systemctl` or on system start if the service is set to automatically start.
* `ExecReload` directive executes when a service restarts. 
* `ExecStop`, `ExecStopPre`, and `ExecStopPost` directives execute when a service is stopped.  

Adversaries have created new service files, altered the commands a `.service` file’s directive executes, and modified the user directive a `.service` file executes as, which could result in privilege escalation. Adversaries may also place symbolic links in these directories, enabling systemd to find these payloads regardless of where they reside on the filesystem.(Citation: Anomali Rocke March 2019)(Citation: airwalk backdoor unix systems)(Citation: Rapid7 Service Persistence 22JUNE2016) 

## Atomic Tests

### Atomic Test #1 - Create Systemd ServiceThis test creates a Systemd service unit file and enables it as a service.
**Supported Platforms:** linux#### Attack Commands: Run with `bash`
```bash
echo "[Unit]" > /etc/systemd/system/art-systemd-service.service
echo "Description=Atomic Red Team Systemd Service" >> /etc/systemd/system/art-systemd-service.service
echo "" >> /etc/systemd/system/art-systemd-service.service
echo "[Service]" >> /etc/systemd/system/art-systemd-service.service
echo "Type=simple"
echo "ExecStart=/bin/touch /tmp/art-systemd-execstart-marker" >> /etc/systemd/system/art-systemd-service.service
echo "ExecStartPre=/bin/touch /tmp/art-systemd-execstartpre-marker" >> /etc/systemd/system/art-systemd-service.service
echo "ExecStartPost=/bin/touch /tmp/art-systemd-execstartpost-marker" >> /etc/systemd/system/art-systemd-service.service
echo "ExecReload=/bin/touch /tmp/art-systemd-execreload-marker" >> /etc/systemd/system/art-systemd-service.service
echo "ExecStop=/bin/touch /tmp/art-systemd-execstop-marker" >> /etc/systemd/system/art-systemd-service.service
echo "ExecStopPost=/bin/touch /tmp/art-systemd-execstoppost-marker" >> /etc/systemd/system/art-systemd-service.service
echo "" >> /etc/systemd/system/art-systemd-service.service
echo "[Install]" >> /etc/systemd/system/art-systemd-service.service
echo "WantedBy=default.target" >> /etc/systemd/system/art-systemd-service.service
systemctl daemon-reload
systemctl enable art-systemd-service.service
systemctl start art-systemd-service.service
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 1

#### Cleanup: 
```bash
systemctl stop art-systemd-service.service
systemctl disable art-systemd-service.service
rm -rf /etc/systemd/system/art-systemd-service.service
systemctl daemon-reload
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create SysV ServiceThis test creates a SysV service unit file and enables it as a service.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
echo '#\!/bin/sh' > /usr/local/etc/rc.d/art-test
echo ' ' >> /usr/local/etc/rc.d/art-test
echo '#' >> /usr/local/etc/rc.d/art-test
echo '# PROVIDE: art-test' >> /usr/local/etc/rc.d/art-test
echo '# REQUIRE: LOGIN' >> /usr/local/etc/rc.d/art-test
echo '# KEYWORD: shutdown' >> /usr/local/etc/rc.d/art-test
echo ' ' >> /usr/local/etc/rc.d/art-test
echo '. /etc/rc.subr' >> /usr/local/etc/rc.d/art-test
echo ' ' >> /usr/local/etc/rc.d/art-test
echo 'name="art_test"' >> /usr/local/etc/rc.d/art-test
echo 'rcvar=art_test_enable' >> /usr/local/etc/rc.d/art-test
echo 'load_rc_config ${name}' >> /usr/local/etc/rc.d/art-test
echo 'command="/usr/bin/touch"' >> /usr/local/etc/rc.d/art-test
echo 'start_cmd="art_test_start"' >> /usr/local/etc/rc.d/art-test
echo '' >> /usr/local/etc/rc.d/art-test
echo 'art_test_start()' >> /usr/local/etc/rc.d/art-test     
echo '{' >> /usr/local/etc/rc.d/art-test
echo '  ${command} /tmp/art-test.marker' >> /usr/local/etc/rc.d/art-test
echo '}' >> /usr/local/etc/rc.d/art-test
echo ' ' >> /usr/local/etc/rc.d/art-test     
echo 'run_rc_command "$1"' >> /usr/local/etc/rc.d/art-test
chmod +x /usr/local/etc/rc.d/art-test
service art-test enable
service art-test start
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 2

#### Cleanup: 
```sh
sysrc -x art_test_enable
rm -f /usr/local/etc/rc.d/art-test
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create Systemd Service file,  Enable the service , Modify and Reload the service.
This test creates a systemd service unit file and enables it to autostart on boot. Once service is created and enabled, it also modifies this same service file showcasing both Creation and Modification of system process. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: System must be Ubuntu ,Kali OR CentOS.

##### Check Prereq Commands:
```bash
if [ $(cat /etc/os-release | grep -i ID=ubuntu) ] || [ $(cat /etc/os-release | grep -i ID=kali) ] || [ $(cat /etc/os-release | grep -i 'ID="centos"') ]; then exit /b 0; else exit /b 1; fi;

```
##### Get Prereq Commands:
```bash
echo Please run from Ubuntu ,Kali OR CentOS.

```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
cat > /etc/init.d/T1543.002 << EOF
#!/bin/bash
### BEGIN INIT INFO
# Provides : Atomic Test T1543.002
# Required-Start: $all
# Required-Stop : 
# Default-Start: 2 3 4 5
# Default-Stop: 
# Short Description: Atomic Test for Systemd Service Creation
### END INIT INFO
python3 -c "import os, base64;exec(base64.b64decode('aW1wb3J0IG9zCm9zLnBvcGVuKCdlY2hvIGF0b21pYyB0ZXN0IGZvciBDcmVhdGluZyBTeXN0ZW1kIFNlcnZpY2UgVDE1NDMuMDAyID4gL3RtcC9UMTU0My4wMDIuc3lzdGVtZC5zZXJ2aWNlLmNyZWF0aW9uJykK'))"
EOF

chmod +x /etc/init.d/T1543.002
if [ $(cat /etc/os-release | grep -i ID=ubuntu) ] || [ $(cat /etc/os-release | grep -i ID=kali) ]; then update-rc.d T1543.002 defaults; elif [ $(cat /etc/os-release | grep -i 'ID="centos"') ]; then chkconfig T1543.002 on ; else echo "Please run this test on Ubnutu , kali OR centos" ; fi ;
systemctl enable T1543.002
systemctl start T1543.002

echo "python3 -c \"import os, base64;exec(base64.b64decode('aW1wb3J0IG9zCm9zLnBvcGVuKCdlY2hvIGF0b21pYyB0ZXN0IGZvciBtb2RpZnlpbmcgYSBTeXN0ZW1kIFNlcnZpY2UgVDE1NDMuMDAyID4gL3RtcC9UMTU0My4wMDIuc3lzdGVtZC5zZXJ2aWNlLm1vZGlmaWNhdGlvbicpCg=='))\"" | sudo tee -a /etc/init.d/T1543.002
systemctl daemon-reload
systemctl restart T1543.002
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 3

#### Cleanup: 
```bash
systemctl stop T1543.002
systemctl disable T1543.002
rm -rf /etc/init.d/T1543.002
systemctl daemon-reload
```

In [ ]:
Invoke-AtomicTest T1543.002 -TestNumbers 3 -Cleanup

## Detection
Monitor file creation and modification events of Systemd service unit configuration files in the default directory locations for `root` & `user` level permissions. Suspicious processes or scripts spawned in this manner will have a parent process of ‘systemd’, a parent process ID of 1, and will usually execute as the `root` user.(Citation: lambert systemd 2022) 

Suspicious systemd services can also be identified by comparing results against a trusted system baseline. Malicious systemd services may be detected by using the systemctl utility to examine system wide services: `systemctl list-units -–type=service –all`. Analyze the contents of `.service` files present on the file system and ensure that they refer to legitimate, expected executables, and symbolic links.(Citation: Berba hunting linux systemd)

Auditing the execution and command-line arguments of the `systemctl` utility, as well related utilities such as `/usr/sbin/service` may reveal malicious systemd service execution.